<a href="https://colab.research.google.com/github/w1767/w1767/blob/main/Text_Preprocessing_for_social_media_data_Version_2_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [1]:
import pandas as pd    #pandas 
import numpy as np     # numpy
import glob
#visulaization library
import matplotlib.pyplot as plt
import seaborn as sns
#to display all rows and columns
pd.set_option("display.max_rows", None, "display.max_columns", None)

#Spacy text processing 
import spacy
import en_core_web_sm

### Data loading

In [ ]:
path = r'C:\Users\user\Desktop\FiverProject\Project1' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
frame.head()

ValueError: ignored

In [ ]:
print(frame.shape)
frame['TweetID'].nunique()

NameError: ignored

In [ ]:
frame['TweetDate'].unique()

NameError: ignored

In [ ]:
frame['screen_name'].nunique()

NameError: ignored

## Pre-processing for NLP with spaCy

#### Bag of words

##### Bag of words is a way to represent text data numerically. Text data is essentially split into words (or more accurately, tokens), which are features. The frequency of each word in each text data is are the corresponding feature values. For example, we might represent "I love cake very very much" as a bag of words dictionary:

{
'I':1,
'love':1,
'cake':1,
'very':2,
'much':1
}

#### Tokenization

##### Tokenization breaks text data up into its tokens at the level of NLP (word, phrase, sentence). The lowest (and most common) is words, which fits perfectly with our bag of words representation. However, these tokens could also include punctuation, stop words, and other custom tokens. We’ll consider these in the context of tweets and the challenge in the next session.

#### Stemming

##### Stemming refers to truncating words of their affixes (prefixes or suffixes) to approximate them to their root form. This is often done with a lookup dictionary of prefixes and suffixes, making it computationally fast. However, there is a performance trade-off. In the English language, some affixes change the meaning of the word completely, resulting in inaccurate feature representation.

#### Lemmatization

##### The alternative to stemming is lemmatization, where words are reduced to their lemmas, or root form. This is done using a lookup dictionary of words and their lemmas, hence resulting in it being more computationally expensive. However, performance is often better, since features are represented more accurately.


##### Given the relatively smaller size of our dataset, we will use lemmatization.5

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
# Let's see what spaCy does with numbers, contractions, #hashtags, @mentions and URLs
s = "2020 can't get any worse #ihate2020 @bestfriend <https://t.co>"
doc = nlp(s)
# Let's look at the lemmas and is stopword of each token
print(f"Token\\t\\tLemma\\t\\tStopword")
print("="*40)
for token in doc:
    print(f"{token}\\t\\t{token.lemma_}\\t\\t{token.is_stop}")

Token\t\tLemma\t\tStopword
2020\t\t2020\t\tFalse
ca\t\tca\t\tTrue
n't\t\tn't\t\tTrue
get\t\tget\t\tTrue
any\t\tany\t\tTrue
worse\t\tbad\t\tFalse
#\t\t#\t\tFalse
ihate2020\t\tihate2020\t\tFalse
@bestfriend\t\t@bestfriend\t\tFalse
<\t\t<\t\tFalse
https://t.co\t\thttps://t.co\t\tFalse
>\t\t>\t\tFalse


In [ ]:
# We want to also keep #hashtags as a token, so we will modify the spaCy model's token_match
import re
# Retrieve the default token-matching regex pattern
re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)
# Add #hashtag pattern
re_token_match = f"({re_token_match}|#\\w+)"
nlp.tokenizer.token_match = re.compile(re_token_match).match
# Now let's try again
s = "2020 can't get any worse #ihate2020 @bestfriend <https://t.co>"
doc = nlp(s)
# Let's look at the lemmas and is stopword of each token
print(f"Token\\t\\tLemma\\t\\tStopword")
print("="*40)
for token in doc:
    print(f"{token}\\t\\t{token.lemma_}\\t\\t{token.is_stop}")

Token\t\tLemma\t\tStopword
2020\t\t2020\t\tFalse
ca\t\tca\t\tTrue
n't\t\tn't\t\tTrue
get\t\tget\t\tTrue
any\t\tany\t\tTrue
worse\t\tbad\t\tFalse
#ihate2020\t\t#ihate2020\t\tFalse
@bestfriend\t\t@bestfriend\t\tFalse
<\t\t<\t\tFalse
https://t.co\t\thttps://t.co\t\tFalse
>\t\t>\t\tFalse


#### Pre-processing algorithm

We can then proceed to create a preprocessing algorithm, and put it in a function so it can be called on every tweet in the training set. In the following preprocess function, each tweet:

Is changed to lower case

Is tokenized with our modified spaCy model

Has its lemmas of tokens set unioned with our features set

Has its bag-of-words representation constructed in a dictionary freq

Has its hashtags, mentions and URLs counted

In [ ]:
# Create a pre-process function for each tweet
def preprocess(s, nlp, features):
    """
    Given string s, spaCy model nlp, and set features (lemmas encountered),
    pre-process s and return updated features and bag-of-words representation dict freq
    - changes s to lower-case
    - tokenize s using nlp to create a doc
    - update features with lemmas encountered in s
    - create bag-of-words representation in dict type freq, including counts for hashtags, mentions and URLs
    """
# To lowercase
    s = s.lower()
# Creating a doc with spaCy
    doc = nlp(s)
    lemmas = []
    for token in doc:
        lemmas.append(token.lemma_)
# Union between lemmas and our features set
    features |= set(lemmas)
# Constructing a bag of words for the tweet
    freq = {'#':0,'@':0,'URL':0}
    for word in lemmas:
        freq[str(word)] = 0
    for token in doc: 
        if '#' in str(token): freq['#'] += 1 # Count number of hashtags, regardless of hashtag
        if '@' in str(token): freq['@'] += 1 # Count number of mentions, regardless of mention
        if 'http://' in str(token): freq['URL'] += 1 # Count number of URLs, regardless of URL
        freq[str(token.lemma_)] += 1
        
    return features, freq

In [ ]:
preprocess_df = frame # Duplicate for preprocessing
features = set({'#','@','URL'}) # Using set feature to contain all words (lemmas) seen


##### Using our preprocess function, we'll preprocess every tweet, updating features each time with new lemmas seen. With each tweet, the tweet's bag of words representation freq is also appended to an array of bag of words for each tweet (bow_array).

In [ ]:
# Array bow_array of bow representations for each tweet;
# bow_array[i] is the bow representation for tweet id (i+1)
bow_array = [] 
for i in range(len(preprocess_df)):
    features, freq = preprocess(preprocess_df.iloc[i]['TweetText'],nlp,features)
    bow_array.append(freq)
len(bow_array)

1106

In [ ]:
# Create dataframe for bag of words representation for each tweet
bow = pd.DataFrame('0', columns=features,index=range(len(preprocess_df)))
bow['id']=preprocess_df.index
bow.set_index('id',drop=True,inplace=True)
bow.head()

,retail,https://t.co/mhz9znuumc,create,affect,top,🔥,nick,bleak,course,specie,https://t.co/qcojfa927k,cult,@complexstyle,dm,print,🌳,black,🗞,magic,nature+tech,cre,https://t.co/6bmyyzzqdf,'d,🤍🤎,revenue,particularly,https://t.co/ygzb2fkxsk,too,example,’ve,manufacture,l,think,huge,sustainabi,/,kourtney,implement,pair,sad,https://t.co/lerbky9paq,trend,reduce,https://t.co/okt4r56lck,gawryc,launch,leather,https://t.co/r80w4yhifq,application,only,vogue,attend,#fashionnew,:),clothe,sharing,capsule,sorry,https://t.co/lpjqifv3dj,🌏,@evaawt,one,much,celebrate,https://t.co/vylfuxk04k,+,'ve,behave,coverage,👗,#pangaia...…,detail,agency,https://t.co/u4gtladryl,recognition,m,#electric,pack,@relondonbiz,like,@gravikylabs,up,creative,about,mean,@amand,https://t.co/snqpdikc56,sensory,limited,hardworke,long,cap,opportunity,#sustainably,sit,how,@ganagenda,focus,girl,live,super,all,compose,https://t.co/dsswkokorv,:,eco,rt,https://t.co/iwzjwp6rav,pick,https://t.co/4oti5gufwv,sustainably,spy,company,3,#forsale,these,https://t.co/5fhalusmxp,🐝,nothing,news,commerce,earth,general,remotely,#inke,wear,micro,,te,what,tailly,graphic,game,few,"#foodforest,",science,factory,https://t.co/c3wiholc4c,tweet,winona,mood,pangaia,line,out,$,material,️sustainable,🚕,#,https://t.co/nqy4ylsnbu,papaya,sep,https://t.co/anred6dnri,https://t.co/qe3q51xndf,untapped,on,award,paola,🚖,p,loungewear,portal,https://t.co/qj5polvwmc,nor,life,letteralmente,@solvemit,campaign,https://t.co/vhutxgjfqp,yesterday,thank,-,of,essential,https://t.co/zb6yzvoqez,love,25,clothing,collaboration,successf,https://t.co/a9wmean4cq,take,carbon,🌸,https://t.co/qdjpc6pepr,size,evolve,founder,examine,mit,berlin,@retailweek,https://t.co/6hg1oclsgo,@stockx,packaging,ca,final,\n,that,join,special,sable,ground,storytelling,okay,k,honor,pm,#earthday2021!,spin,https://t.co/ryugpcoq4q,reputation,fund,#pangaia,happy,amanda,power,🔅,↗,https://t.co/iigpcxe0s3,™,b,be,who,🖋,heart,20th,eye,logo,ajide,https://t.co/3horbo4imn,almost,@internqueen,grape,https://t.co/irrciz3doi,project,lite,conc,https://t.co/vfccgwl29y,....,;,https://t.co/zzcdzg2pz2,disability,act,my,kidswear,accelerate,sweat,determine,#innovation…,buy,#sustainability…,https://t.co/yykxqkld4j,https://t.co/qui3jy2e3o,https://t.co/1mo4sxldhp,will,become,an,another,☘,https://t.co/3o4mt4hgtp,or,once,🌵,september,cotton,https://t.co/sn1evxtfgn,flwrdwn,flower,@kiranforpeace,many,meet,utilize,move,colour,",",’,'s,animal,giulia,shirt,18,scientific,guy,linear,great,t-,🏾,t,campbell,service,🌴,car,genius,but,last,kensington,track,kiva,https://t.co/frjcgbcsn9,https://t.co/mkkahzirab,study,https://t.co/kycjccfznc,🐧,@steinfeldsstyle,snapshot,2013,burst,offer,😂,friendly,',@wemakewebsite,feel,https://t.co/aaz8kvu9ah,,https://t.co/sgi0lv60jb,https://t.co/qfcouib98d,concept,cool,selfridge,@hypebae,#climatetech,https://t.co/pbvivnz9vt,💙,everything,sheeeeesh,https://t.co/peyqgzmaur,such,#naomicampbell,wildflower,toxic,#ladygaga,visual,america,allow,yet,shadow,🧡,💜,o,conceptualiz,understand,down,@toogoodtogo_uk,spinoff,range,rebuild,base,store,here,💞,https://t.co/24goz8otew,@ex_aih,etc,@nylonmag,beyond,no,prefer,ring,endanger,💕,sleeve,shit,streetwear,💖,https://t.co/audsm2h3xn,when,set,@voguemagazine,#selfridgesmanchester,and,https://t.co/6qorl6awgr,🙏,https://t.co/ogtg7z6bkr,health,@habibh,follow,@robinr,@kiva,give,@basque_design,@gr,🧬,#earthday.,🙌,inferno,limit,ass,ethically,would,consumer,🙃,kicking,explore,depend,https://t.co/9j92uteqnc,https://t.co/ejuxbpu81d,su,https://t.co/kxygut2oew,peel,need,so,transphobic,those,sport,intersection,https://t.co/oo4pwuwvsu,coul,@iambiotech,partnership,♻,particle,awesome,@biobean,–,say,pop,innovation,blue,@helpful_card,source,https://t.co/fc3llmapzd,outside,kick,disturb,destination,#pangaiataxis,#kuwtk,https://t.co/cpaiigaaed,https://t.co/wto6n7o5ph,hit,shopping,✨,diesel,first,gone,@takashipom,between,flwdwn,very,…,@touchaddict,still,emission,https://t.co/3di8toeppk,change,?,@medialab,ill,color,co2,sustainability,neat,#sweatpant,make,expe,dark,ht

In [ ]:
# Update bow[i] with bag-of-words freq of the tweet id (i+1)
for i in range(len(preprocess_df)):
    freq = bow_array[i]
    for f in freq:
        bow.loc[i+1,f]=freq[f]
bow.head()

,retail,https://t.co/mhz9znuumc,create,affect,top,🔥,nick,bleak,course,specie,https://t.co/qcojfa927k,cult,@complexstyle,dm,print,🌳,black,🗞,magic,nature+tech,cre,https://t.co/6bmyyzzqdf,'d,🤍🤎,revenue,particularly,https://t.co/ygzb2fkxsk,too,example,’ve,manufacture,l,think,huge,sustainabi,/,kourtney,implement,pair,sad,https://t.co/lerbky9paq,trend,reduce,https://t.co/okt4r56lck,gawryc,launch,leather,https://t.co/r80w4yhifq,application,only,vogue,attend,#fashionnew,:),clothe,sharing,capsule,sorry,https://t.co/lpjqifv3dj,🌏,@evaawt,one,much,celebrate,https://t.co/vylfuxk04k,+,'ve,behave,coverage,👗,#pangaia...…,detail,agency,https://t.co/u4gtladryl,recognition,m,#electric,pack,@relondonbiz,like,@gravikylabs,up,creative,about,mean,@amand,https://t.co/snqpdikc56,sensory,limited,hardworke,long,cap,opportunity,#sustainably,sit,how,@ganagenda,focus,girl,live,super,all,compose,https://t.co/dsswkokorv,:,eco,rt,https://t.co/iwzjwp6rav,pick,https://t.co/4oti5gufwv,sustainably,spy,company,3,#forsale,these,https://t.co/5fhalusmxp,🐝,nothing,news,commerce,earth,general,remotely,#inke,wear,micro,,te,what,tailly,graphic,game,few,"#foodforest,",science,factory,https://t.co/c3wiholc4c,tweet,winona,mood,pangaia,line,out,$,material,️sustainable,🚕,#,https://t.co/nqy4ylsnbu,papaya,sep,https://t.co/anred6dnri,https://t.co/qe3q51xndf,untapped,on,award,paola,🚖,p,loungewear,portal,https://t.co/qj5polvwmc,nor,life,letteralmente,@solvemit,campaign,https://t.co/vhutxgjfqp,yesterday,thank,-,of,essential,https://t.co/zb6yzvoqez,love,25,clothing,collaboration,successf,https://t.co/a9wmean4cq,take,carbon,🌸,https://t.co/qdjpc6pepr,size,evolve,founder,examine,mit,berlin,@retailweek,https://t.co/6hg1oclsgo,@stockx,packaging,ca,final,\n,that,join,special,sable,ground,storytelling,okay,k,honor,pm,#earthday2021!,spin,https://t.co/ryugpcoq4q,reputation,fund,#pangaia,happy,amanda,power,🔅,↗,https://t.co/iigpcxe0s3,™,b,be,who,🖋,heart,20th,eye,logo,ajide,https://t.co/3horbo4imn,almost,@internqueen,grape,https://t.co/irrciz3doi,project,lite,conc,https://t.co/vfccgwl29y,....,;,https://t.co/zzcdzg2pz2,disability,act,my,kidswear,accelerate,sweat,determine,#innovation…,buy,#sustainability…,https://t.co/yykxqkld4j,https://t.co/qui3jy2e3o,https://t.co/1mo4sxldhp,will,become,an,another,☘,https://t.co/3o4mt4hgtp,or,once,🌵,september,cotton,https://t.co/sn1evxtfgn,flwrdwn,flower,@kiranforpeace,many,meet,utilize,move,colour,",",’,'s,animal,giulia,shirt,18,scientific,guy,linear,great,t-,🏾,t,campbell,service,🌴,car,genius,but,last,kensington,track,kiva,https://t.co/frjcgbcsn9,https://t.co/mkkahzirab,study,https://t.co/kycjccfznc,🐧,@steinfeldsstyle,snapshot,2013,burst,offer,😂,friendly,',@wemakewebsite,feel,https://t.co/aaz8kvu9ah,,https://t.co/sgi0lv60jb,https://t.co/qfcouib98d,concept,cool,selfridge,@hypebae,#climatetech,https://t.co/pbvivnz9vt,💙,everything,sheeeeesh,https://t.co/peyqgzmaur,such,#naomicampbell,wildflower,toxic,#ladygaga,visual,america,allow,yet,shadow,🧡,💜,o,conceptualiz,understand,down,@toogoodtogo_uk,spinoff,range,rebuild,base,store,here,💞,https://t.co/24goz8otew,@ex_aih,etc,@nylonmag,beyond,no,prefer,ring,endanger,💕,sleeve,shit,streetwear,💖,https://t.co/audsm2h3xn,when,set,@voguemagazine,#selfridgesmanchester,and,https://t.co/6qorl6awgr,🙏,https://t.co/ogtg7z6bkr,health,@habibh,follow,@robinr,@kiva,give,@basque_design,@gr,🧬,#earthday.,🙌,inferno,limit,ass,ethically,would,consumer,🙃,kicking,explore,depend,https://t.co/9j92uteqnc,https://t.co/ejuxbpu81d,su,https://t.co/kxygut2oew,peel,need,so,transphobic,those,sport,intersection,https://t.co/oo4pwuwvsu,coul,@iambiotech,partnership,♻,particle,awesome,@biobean,–,say,pop,innovation,blue,@helpful_card,source,https://t.co/fc3llmapzd,outside,kick,disturb,destination,#pangaiataxis,#kuwtk,https://t.co/cpaiigaaed,https://t.co/wto6n7o5ph,hit,shopping,✨,diesel,first,gone,@takashipom,between,flwdwn,very,…,@touchaddict,still,emission,https://t.co/3di8toeppk,change,?,@medialab,ill,color,co2,sustainability,neat,#sweatpant,make,expe,dark,ht

In [ ]:
# Join bag-of-words representation to train dataframe
# Append _data suffix to 'keyword','location','text','target' for features that are not lemma tokens
preprocess_df = preprocess_df.join(bow,lsuffix='_data')
# Saving bag-of-words representation for collaborators
#preprocess_df.to_csv("data/train_preprocessed.csv",index=True,index_label='id')
preprocess_df.head()

TweetID            TweetDate  \
0  1387732819175161856  2021-04-29 11:37:41   
1  1387732069283946496  2021-04-29 11:34:42   
2  1387538277448798213  2021-04-28 22:44:39   
3  1386998563968851970  2021-04-27 11:00:01   
4  1385643752300109825  2021-04-23 17:16:29   

                                           TweetText  retweetCount  \
0  RT @PromostylParis: Eco-friendly essential ♻️\...             1   
1  Eco-friendly essential ♻️\nThe eco-friendly br...             1   
2  re-selling blue pangaia tracksuit in size XS  ...             0   
3  Check out the latest #PANGAIA looks modeled by...             0   
4  @techonomy @thepangaia @amandajparkes So much ...             0   

   FavouriteCount  IsFavourite  ISretweetd             userDate  \
0               0        False       False  2010-05-06 23:33:58   
1               0        False       False  2012-11-13 16:37:08   
2               0        False       False  2019-08-24 18:42:55   
3               5        False       False  2008-08-31 16:15:32   
4               0        False       False  2007-03-19 01:27:57   

      screen_name  status_count  follower_count  freind_count  \
0  imagine_garden        244076            5008           993   
1  PromostylParis          7283            2389           554   
2   pangaiakuwait             2               0             0   
3     designscene         35590           15517           294   
4          robinr         17858            5219          3994   

   userFavouriteCount retail_data https://t.co/mhz9znuumc_data create_data  \
0                4099           0                            0           0   
1                1623           0                            0           0   
2                   1           0                            0           0   
3                7130           0                            0           0   
4                3308           0                            0           0   

  affect_data top_data 🔥_data nick_data bleak_data course_data specie_data  \
0           0        0      0         0          0           0           0   
1           0        0      0         0          0           0           0   
2           0        0      0         0          0           0           0   
3           0        0      0         0          0           0           0   
4           0        0      0         0          0           0           0   

  https://t.co/qcojfa927k_data cult_data @complexstyle_data dm_data  \
0                            0         0                  0       0   
1                            0         0                  0       0   
2                            0         0                  0       0   
3                            0         0                  0       1   
4                            0         0                  0       0   

  print_data 🌳_data black_data 🗞_data magic_data nature+tech_data cre_data  \
0          0      0          0      0          0                0        0   
1          0      0          0      0          0                0        0   
2          0      0          0      0          0                0        0   
3          0      0          0      0          0                0        0   
4          0      0          0      0          0                0        0   

  https://t.co/6bmyyzzqdf_data 'd_data 🤍🤎_data revenue_data particularly_data  \
0                            0       0       0            0                 0   
1                            0       0       0            0                 0   
2                            0       0       0            0                 0   
3                            0       0       0            0                 0   
4                            0       0       0            0                 0   

  https://t.co/ygzb2fkxsk_data too_data example_data ’ve_data  \
0                            0        0            0        0   
1                            0        0            0        0   
2